In [32]:
import matplotlib.pyplot as plt
from desispec.io.spectra import read_spectra
import numpy as np
from desitarget.targetmask import desi_mask
# Load the coadded spectra
spec = read_spectra("coadd-main-dark-0.fits")

# Inspect the available arms (b, r, z)
print("Arms available:", spec.wave.keys())  # usually: 'b', 'r', 'z'

# Let's choose one arm to plot (e.g. 'r' = red)
arm = 'z'
print(spec.flux[arm].shape)  # shape of the flux array
print(spec.ivar[arm].shape)  # shape of the ivar array
print(spec.wave[arm].shape)  # shape of the mask array
fibermap = spec.fibermap

# Boolean mask for QSOs
is_qso = (fibermap['DESI_TARGET'] & desi_mask.QSO) != 0

# Print how many QSOs are in the file
print(f"Number of QSOs: {is_qso.sum()}")

INFO:spectra.py:451:read_spectra: iotime 0.402 sec to read spectra from:  coadd-main-dark-0.fits at 2025-05-08T21:10:32.335520
Arms available: dict_keys(['b', 'r', 'z'])
(1704, 2881)
(1704, 2881)
(2881,)
Number of QSOs: 271


In [ ]:

# Pick the first target in the spectra (index 0)
flux = spec.flux[arm][0]
wave = spec.wave[arm]
ivar = spec.ivar[arm][0]  # Inverse variance

# Optional: mask bad pixels using the inverse variance
good = ivar > 0
wave = wave[good]
flux = flux[good]

# Plot the spectrum
plt.figure(figsize=(10, 5))
plt.plot(wave, flux, label=f'Arm: {arm.upper()}', lw=0.8)
plt.xlabel("Wavelength [Å]")
plt.ylabel("Flux [1e-17 erg/s/cm²/Å]")
plt.title("DESI Coadded Spectrum")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
